In [ ]:
import pandas as pd
import duckdb as ddb
import datetime
import glob
import os
import re
import matplotlib.pyplot as plt

In [ ]:
dir= "G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\row_files\\for_combination\\"
#dir="C:/#D von 65456/Neuer Ordner/"
temp_file=''
log_file= '' 

In [ ]:
df_measures_date_format= "%Y-%m-%d %H:%M:%S" #"%Y.%m.%d %H:%M:%S"# "2022-02-25 15:30:00"
df_log_date_format= "%Y/%m/%d %H:%M:%S"

In [ ]:
def dateparse (time_str):    
    return pd.to_datetime(time_str)

In [ ]:
def handle_special_numbers(df: pd.DataFrame, columns_list: list):
    for column in columns_list:
        new_column= df[column].str.replace(pat='\+( )+',repl= '+',regex= True).str.replace(pat= '-( )+', repl= '-',regex= True)
        df[column]= pd.to_numeric(new_column)
    return df

In [ ]:
#Apply shift and cut to the signal
def shift_cut(df:pd.DataFrame):
    Z_TCP_OK = 0
    Z_Schlitten_OK = 0
    Z_Hals_OK = 0
    Z_Welle_OK = 0
    n= 4
    df['z_tcp_ok_orginal']= df['z_tcp_ok'].copy()
    for index, row in df.iterrows():
        if row["drhz"] >=3000:
            ###Get the values N steps before. This should be the 0 value
            Z_TCP_OK= df['z_tcp_ok'][index - n]
            Z_Schlitten_OK = df['z_schlitten_ok'][index - n]
            Z_Hals_OK = df['z_hals_ok'][index - n]
            Z_Welle_OK = df['z_welle_ok'][index - n]
            ## Apply Shift
            df['z_tcp_ok']= df['z_tcp_ok'] - Z_TCP_OK
            df['z_schlitten_ok']= df['z_schlitten_ok'] - Z_Schlitten_OK
            df['z_hals_ok']= df['z_hals_ok'] - Z_Hals_OK
            df['z_welle_ok']= df['z_welle_ok'] - Z_Welle_OK
            ## Apply cut
            df= df[index -n :]
            break
    return df

In [ ]:
for current_dir in os.listdir(dir):
    if os.path.isdir(os.path.join(dir, current_dir)):
        print (current_dir)
        os.chdir(os.path.join(dir, current_dir))
    working_dir= str(dir+ current_dir+'\\')
    #### Find relevant measure file#####
    temp_file= None
    for file in glob.glob('_N_*MPF.xlsx'):
        if file is not None:
            temp_file= file
            break

    #### Find relevant log file#####
    log_file= None
    for file in glob.glob('*.CSV'):
        if re.match(pattern='^\d.*\d\.CSV$',string= file) is not None:
                log_file= file
    if temp_file is None or log_file is None:
        continue

    print ("First File: ", temp_file)
    print ("Second File: ", log_file)
    ######Load file with 2 minutes sampling rate
    df_measures= pd.read_excel(str(working_dir+temp_file),sheet_name=0, parse_dates= True, date_parser=dateparse)
    df_measures['DAT_UHZ']= df_measures['DAT_UHZ'].apply(lambda t: t.replace(second=0))
    to_drop= ['T_Bett', 'T_Spindel', 'DATUM', 'ZEIT','DAT_UHZ.1', 'count', 'Z_Schlitten_OK', 'Z_Hals_OK']
    for column in df_measures.columns:
        if str(column).find('Unnamed') != -1:
            to_drop.append(column)
    df_measures.drop(columns=to_drop, inplace=True)
    df_measures.columns= df_measures.columns.str.lower()
    df_measures_fields=  df_measures.columns.to_list()
    print('Resulted fields: ')
    print(df_measures_fields)
    fields= ""
    for field in df_measures_fields:
        if fields == "":
            fields = "df_measures."+field
        else:
            fields += ", df_measures."+ field
    df_measures_fields= fields
#    print(df_measures.head(3))
    file_path=str(dir+'output\\'+current_dir+'.csv')
    df_measures['M8']= 1
    df_measures['M121']= 0
    df_measures['M127']= 0
    df_measures['M7']= 0
    df_measures.to_csv(file_path)
    #####Load log file with 20 seconds sampling rate
    content = open(str(working_dir+log_file)).readlines()
    lines = [line_num for line_num, line_content in enumerate(content) if "Data" in line_content]
    skipped_rows= list(range(0,lines[0]+1))
    skipped_rows.append(lines[0]+2)
    df_logs= pd.read_csv(str(working_dir+log_file), sep=',',skiprows= skipped_rows, infer_datetime_format= True, parse_dates= True, date_parser= dateparse)
    # CH4=Vorlauftemperatur und CH3=Rücklauftemperatur (in der Datei steht es falsch herum)
    # CH5=Volumenstrom kann man zur Dokumentationszwecken mitnehmen ist aber nicht notwendig
    # Die Alarme haben keine Funktion und könnnen weglassen werden
    df_logs.drop(columns=["Alarm1-10","Alarm11-20","AlarmOut","ms","Number"], inplace=True)
    df_logs.rename(columns={'Date&Time': 'time_stamp','CH1': 'T_KSS_TANK', 'CH2':'T_RAUM', 'CH3':'T_SPINDEL_RUECK', 'CH4':'T_SPINDEL_Vor', 'CH5':'Volumenstrom'}, inplace= True)#, 'CH6':"T_Schwenkantrieb_Flansch"
    df_logs.columns= df_logs.columns.str.lower()
    print(df_logs.columns)
    df_logs= handle_special_numbers(df_logs, columns_list=['t_kss_tank', 't_raum', 't_spindel_vor','t_spindel_rueck', 'volumenstrom'])
    df_logs_fields= df_logs.columns.to_list()
    fields= ""
    for field in df_logs_fields:
        if fields == "":
            fields = "df_logs." + field
        else:
            fields += ", df_logs." + field
    df_logs_fields= fields
    ####Apply Query
    query= """
    select * from 
    (
    select df_measures.start as start, df_measures.end as end,{df_measures_columns}, min(df_logs.ts) as ts from 
        (
        (select  {df_measures_columns}, strptime(df_measures.dat_uhz,'{df_m_date_format}') as start, (strptime(df_measures.dat_uhz,'{df_m_date_format}') + interval 2 Minute) as end   from df_measures) as df_measures
        join
        (select {df_logs_columns}, strptime(df_logs.time_stamp ,'{log_date_format}') as ts from df_logs) as df_logs
        on (df_logs.ts >= df_measures.start) and ( df_logs.ts < df_measures.end)
        )
    group by df_measures.start, df_measures.end, {df_measures_columns}
    ) combined_logs
    join 
    df_logs
    on (combined_logs.ts = strptime(df_logs.time_stamp ,'{log_date_format}'))
    order by start Asc
    """.format(df_measures_columns= df_measures_fields, df_logs_columns= df_logs_fields, df_m_date_format= df_measures_date_format, log_date_format= df_log_date_format)
    results= ddb.query(query=query)
    result_df= results.to_df()
    
    ###Insert cooling data
    ###M8
    if current_dir.find('_M8')== -1:
        result_df['M8']= 0
    else:
        result_df['M8']= 1
    ###M121
    if current_dir.find('_M121')== -1:
        result_df['M121']= 0
    else:
        result_df['M121']= 1
    ###M127
    if current_dir.find('_M127')== -1:
        result_df['M127']= 0
    else:
        result_df['M127']= 1
    ###M7
    if current_dir.find('_M7')== -1:
        result_df['M7']= 0
    else:
        result_df['M7']= 1
    result_df['t_kuehlung']= result_df['t_spindel_vor']
    result_df= shift_cut(result_df)
    result_df.reset_index(inplace= True)
    #result_df.drop(columns= ['level_0', 'index'], inplace= True)
    plt.plot(result_df['z_welle_ok'], label= 'z_wellle_ok')
    plt.grid()
    plt.show()
    plt.plot(result_df['drhz'], label= 'DRHZ')
    plt.grid()
    plt.show()
    #file_path=str(dir+current_dir+'\\'+current_dir+'.csv')
    #file_path=str(dir+'output\\'+current_dir+'.csv')
    #result_df.to_csv(file_path)
    print('Result File saved ', file_path)
    print ('-----------------------------------------')